In [1]:
#Not all of these imports are actually used in the current iteration of this notebook but may be used in the future
import tweepy as tw
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import integrate
import pandas as pd
import collections
import threading
from datetime import datetime, timedelta
from customFunctions import *
import operator
import csv
from datetime import datetime
import os

## Setting an OS path for writing data and opening directories (machine dependent):

In [3]:
#CHANGE FOR EACH DATA SCRAPE RUN
parent_dir = "Yours here"
#parent_dir = "Yours here"

## Creating an api instance for Tweepy:

In [4]:
#Enter your own Twitter API credentials here
auth = tw.AppAuthHandler("Yours here", "Yours here")
api = tw.API(auth, wait_on_rate_limit=True)

## Creating arrays with info for each MC pulled from 116thCongressTwitter.csv:

In [21]:
#Setting up the arrays that hold data for members by party and chamber
HouseR = [[]]
HouseD = [[]]
HouseO = [[]]
SenateR = [[]]
SenateD = [[]]
SenateO = [[]]

with open('LookupSheets/116thCongressTwitter.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')

        for row in csv_reader:
            if(not (row[7] == "PR" or row[7] == "DC" or row[7] == "USVI" or row[7] == "NMI" or row[7] == "GU")):
                # official screenname, campaign screenname, name, party code, district (if Rep), state icps, state, year born, dw nom dim 1, dw nom dim 2
                info = [row[2], row[3], row[0], row[8], row[6], row[4], row[7], row[9], row[10], row[11]]

                if(row[1] == "Rep"):
                    if(row[8] == "200" or row[8] == "331"): HouseR = HouseR + [info]
                    elif(row[8] == "100" or row[8] == "329"): HouseD = HouseD + [info]
                    else: HouseO = HouseO + [info]

                if(row[1] == "Sen"):
                    if(row[8] == "200" or row[8] == "331"): SenateR = SenateR + [info]
                    elif(row[8] == "100" or row[8] == "329"): SenateD = SenateD + [info]
                    else: SenateO = SenateO + [info]
                    
HouseR = HouseR[1:]
HouseD = HouseD[1:]
HouseO = HouseO[1:]
SenateR = SenateR[1:]
SenateD = SenateD[1:]
SenateO = SenateO[1:]

## Creating the file structure for data to be stored in
(only needs to be run once per dataset collection and/or device)

In [22]:
with open('LookupSheets/116thCongressTwitter.csv') as csv_file:
    
    csv_reader = csv.reader(csv_file, delimiter=',')
    
    for row in csv_reader:
        if(row[1] == "Rep" and not (row[7] == "PR" or row[7] == "DC" or row[7] == "USVI" 
                                 or row[7] == "NMI" or row[7] == "GU")):
    
            
            #Index values correlate with column values in 116thCongressTwitterCodebook.rtf 
            info = [row[0], row[2], row[3], row[4], row[5], row[6], 
                    row[7], row[8], row[9], row[10], row[11], row[12], row[13]]
            
            
            directory = row[7] + '_' + row[6] + '_' + row[0].replace(" ", "")
            
            path = os.path.join(parent_dir, directory)
            os.mkdir(path, 0o666)

            filepath = parent_dir + '/' + directory + '/' + directory + '.csv'
            with open(filepath, 'w') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
                filewriter.writerow(info)
                
            current_dir = parent_dir + '/' + directory
            
            directory_campaign = 'Campaign'
            
            directory_official = 'Official'
            
            path = os.path.join(current_dir, directory_campaign)
            os.mkdir(path, 0o666)
            
            path = os.path.join(current_dir, directory_official)
            os.mkdir(path, 0o666)
            
            open(current_dir + '/' + directory_campaign + "/.gitkeep", "w+")
            open(current_dir + '/' + directory_official + "/.gitkeep", "w+")


        if(row[1] == "Sen"):
            
            #Note to self: CHANGE IF RUNNING ON PI
            parent_dir = "Yours here"
            
            #Index values correlate with column values in 116thCongressTwitterCodebook.rtf 
            info = [row[0], row[2], row[3], row[4], row[5], row[6], 
                    row[7], row[8], row[9], row[10], row[11], row[12], row[13]]
            
            
            directory = row[7] + '_' + row[0].replace(" ", "")
            
            path = os.path.join(parent_dir, directory)
            os.mkdir(path, 0o666)

            filepath = parent_dir + '/' + directory + '/' + directory + '.csv'
            with open(filepath, 'w') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
                filewriter.writerow(info)
                
            current_dir = parent_dir + '/' + directory
            
            directory_campaign = 'Campaign'
            
            directory_official = 'Official'
            
            path = os.path.join(current_dir, directory_campaign)
            os.mkdir(path, 0o666)
            
            path = os.path.join(current_dir, directory_official)
            os.mkdir(path, 0o666)
            
            open(current_dir + '/' + directory_campaign + "/.gitkeep", "w+")
            open(current_dir + '/' + directory_official + "/.gitkeep", "w+")

## Scraping the data

In [23]:
t_1 = datetime(2019, 1, 20) # first covid case in the US
t_2 = datetime(2020, 1, 20) # End date of scrape range

timespan = [t_1, t_2]

# Keywords to filter for
# k = ["covid", "corona", "wuhanvirus", "wuhan virus", "china virus", "chinavirus", 
#      "social distance", "socially distant", "socialdistance", "socially-distant", "sociallydistant"]
k=None

# Loop over all House D's accounts and scrape tweets to csv files containing keywords
for info in HouseD:
    filepath = parent_dir + '/House/' + info[6] + '_' + info[4] + '_' + info[2].replace(' ', '')
    if(info[0] != ''): 
        writeTweets(info[0], api, timespan, filepath + '/Official/' + info[0]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
    if(info[1] != ''): 
        writeTweets(info[1], api, timespan, filepath + '/Campaign/' + info[1]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
print("####HOUSE DEMS COMPLETE#####")
print("")

# Loop over all House R's accounts and scrape tweets to csv files containing keywords
for info in HouseR:
    filepath = parent_dir + '/House/' + info[6] + '_' + info[4] + '_' + info[2].replace(' ', '')
    if(info[0] != ''): 
        writeTweets(info[0], api, timespan, filepath + '/Official/' + info[0]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
    if(info[1] != ''): 
        writeTweets(info[1], api, timespan, filepath + '/Campaign/' + info[1]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
print("####HOUSE GOP COMPLETE#####")
print("")

# Loop over all House O's accounts and scrape tweets to csv files containing keywords
for info in HouseO:
    filepath = parent_dir + '/House/' + info[6] + '_' + info[4] + '_' + info[2].replace(' ', '')
    if(info[0] != ''): 
        writeTweets(info[0], api, timespan, filepath + '/Official/' + info[0]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
    if(info[1] != ''): 
        writeTweets(info[1], api, timespan, filepath + '/Campaign/' + info[1]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
print("####HOUSE OTHERS COMPLETE#####")
print("")

# Loop over all Senate D's accounts and scrape tweets to csv files containing keywords
for info in SenateD:
    filepath = parent_dir + '/Senate/' + info[6] + '_' + info[2].replace(' ', '')
    if(info[0] != ''): 
        writeTweets(info[0], api, timespan, filepath + '/Official/' + info[0]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
    if(info[1] != ''): 
        writeTweets(info[1], api, timespan, filepath + '/Campaign/' + info[1]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
print("####SENATE DEMS COMPLETE#####")
print("")

# Loop over all Senate R's accounts and scrape tweets to csv files containing keywords
for info in SenateR:
    filepath = parent_dir + '/Senate/' + info[6] + '_' + info[2].replace(' ', '')
    if(info[0] != ''): 
        writeTweets(info[0], api, timespan, filepath + '/Official/' + info[0]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
    if(info[1] != ''): 
        writeTweets(info[1], api, timespan, filepath + '/Campaign/' + info[1]
                    + '_' + str(timespan[0].date()) + '_to_' + str(timespan[1].date()) + '.csv', keywords = k)
print("####SENATE GOP COMPLETE#####")
print("")

Working on:   RepAdams
.csv opened
----------------------------------
Working on:   AlmaforCongress
.csv opened
----------------------------------
Working on:   reppeteaguilar
.csv opened
----------------------------------
Working on:   AguilarCampaign
.csv opened
----------------------------------
Working on:   RepColinAllred
.csv opened
----------------------------------
Working on:   ColinAllredTX
.csv opened
----------------------------------
Working on:   RepCindyAxne
.csv opened
----------------------------------
Working on:   Axne4Congress
.csv opened
----------------------------------
Working on:   RepBarragan
.csv opened
----------------------------------
Working on:   Nanette4CA
.csv opened
----------------------------------
Working on:   RepKarenBass
.csv opened
----------------------------------
Working on:   KarenBassTweets
.csv opened
----------------------------------
Working on:   RepBeatty
.csv opened
----------------------------------
Working on:   JoyceBeatty
.csv op

.csv opened
----------------------------------
Working on:   RepTedDeutch
.csv opened
----------------------------------
Working on:   RedDeutch
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Working on:   RepDebDingell
.csv opened
----------------------------------
Working on:   DebDingell
.csv opened
----------------------------------
Working on:   RepLloydDoggett
.csv opened
----------------------------------
Working on:   LloydDoggettTX
.csv opened
----------------------------------
Working on:   USRepMikeDoyle
.csv opened
----------------------------------
Working on:   rep_doyle
.csv opened
----------------------------------
Working on:   RepEliotEngel
.csv opened
----------------------------------
Working on:   TeamEngel
.csv opened
----------------------------------
Working on:   RepEscobar
.csv opened
----------------------------------
Working on:   vgescobar
.csv opened
----------------------------------
Working on:   RepAnnaEshoo
.csv opened
------------------

.csv opened
----------------------------------
Working on:   ConorLambPA
.csv opened
----------------------------------
Working on:   JimLangevin
.csv opened
----------------------------------
Working on:   LangevinForRI
.csv opened
----------------------------------
Working on:   RepRickLarsen
.csv opened
----------------------------------
Working on:   larsenrick
.csv opened
----------------------------------
Working on:   RepJohnLarson
.csv opened
----------------------------------
Working on:   JohnLarsonCT
.csv opened
----------------------------------
Working on:   RepLawrence
.csv opened
----------------------------------
Working on:   BrendaLLawrence
.csv opened
----------------------------------
Working on:   RepAlLawsonJr
.csv opened
----------------------------------
Working on:   AlLawsonJr
.csv opened
----------------------------------
Working on:   RepBarbaraLee
.csv opened
----------------------------------
Working on:   BLeeForCongress
.csv opened
----------------------

.csv opened
----------------------------------
Working on:   RepMarkPocan
.csv opened
----------------------------------
Working on:   MarkRocan
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Working on:   RepKatiePorter
.csv opened
----------------------------------
Working on:   katieporteroc
.csv opened
----------------------------------
Working on:   RepPressley
.csv opened
----------------------------------
Working on:   AyannaPressley
.csv opened
----------------------------------
Working on:   RepDavidEPrice
.csv opened
----------------------------------
Working on:   DPrice4Congress
.csv opened
----------------------------------
Working on:   RepMikeQuigley
.csv opened
----------------------------------
Working on:   QuigleyCampaign
.csv opened
----------------------------------
Working on:   RepRaskin
.csv opened
----------------------------------
Working on:   jamie_raskin
.csv opened
----------------------------------
Working on:   RepKathleenRice
.csv opened


.csv opened
----------------------------------
Working on:   RepVisclosky
.csv opened
----------------------------------
Working on:   TeamVisclosky
.csv opened
----------------------------------
Working on:   RepDWStweets
.csv opened
----------------------------------
Working on:   DWStweets
.csv opened
----------------------------------
Working on:   RepMaxineWaters
.csv opened
----------------------------------
Working on:   MaxineWaters
.csv opened
----------------------------------
Working on:   RepBonnie
.csv opened
----------------------------------
Working on:   Bonnie4Congress
.csv opened
----------------------------------
Working on:   PeterWelch
.csv opened
----------------------------------
Working on:   WelchForVT
.csv opened
----------------------------------
Working on:   RepWexton
.csv opened
----------------------------------
Working on:   JenniferWexton
.csv opened
----------------------------------
Working on:   RepSusanWild
.csv opened
------------------------------

.csv opened
----------------------------------
Working on:   Duncan4Congress
.csv opened
----------------------------------
Working on:   drnealdunnfl2
.csv opened
----------------------------------
Working on:   DunnCampaign
.csv opened
----------------------------------
Working on:   RepTomEmmer
.csv opened
----------------------------------
Working on:   tomemmer
.csv opened
----------------------------------
Working on:   RepRonEstes
.csv opened
----------------------------------
Working on:   RonEsteKS
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Working on:   RepDrewFerguson
.csv opened
----------------------------------
Working on:   DrewFergusonGA
.csv opened
----------------------------------
Working on:   repbrianfitz
.csv opened
----------------------------------
Working on:   BrianFitzUSA
.csv opened
----------------------------------
Working on:   RepChuck
.csv opened
----------------------------------
Working on:   chuck4congress
.csv opened
-------------

.csv opened
----------------------------------
Working on:   RepLaMalfa
.csv opened
----------------------------------
Working on:   DougLaMalfa
Not authorized.
Working on:   RepDLamborn
.csv opened
----------------------------------
Working on:   boblatta
.csv opened
----------------------------------
Working on:   latta4congress
.csv opened
----------------------------------
Working on:   RepDLesko
.csv opened
----------------------------------
Working on:   DebbieLesko
.csv opened
----------------------------------
Working on:   USRepLong
.csv opened
----------------------------------
Working on:   auctnr1
.csv opened
----------------------------------
Working on:   RepLoudermilk
.csv opened
----------------------------------
Working on:   standwithbarry
.csv opened
----------------------------------
Working on:   RepFrankLucas
.csv opened
----------------------------------
Working on:   FrankDLucas
.csv opened
----------------------------------
Working on:   RepBlaine
.csv opened
-

.csv opened
----------------------------------
Working on:   RepPeteStauber
.csv opened
----------------------------------
Working on:   PeteStauber
.csv opened
----------------------------------
Working on:   RepStefanik
.csv opened
----------------------------------
Working on:   EliseStefanik
.csv opened
----------------------------------
Working on:   RepBryanSteil
.csv opened
----------------------------------
Working on:   BryanSteil
.csv opened
----------------------------------
Working on:   RepGregSteube
.csv opened
----------------------------------
Working on:   gregsteube
.csv opened
----------------------------------
Working on:   RepChrisStewart
.csv opened
----------------------------------
Working on:   StewartforUtah
.csv opened
----------------------------------
Working on:   RepSteveStivers
.csv opened
----------------------------------
Working on:   stevestivers
.csv opened
----------------------------------
Working on:   RepVanTaylor
.csv opened
-------------------

.csv opened
----------------------------------
Working on:   SenAmyKlobuchar
.csv opened
----------------------------------
Working on:   amyklobuchar
.csv opened
----------------------------------
Working on:   SenatorLeahy
.csv opened
----------------------------------
Working on:   PatrickLeahy
.csv opened
----------------------------------
Working on:   Sen_JoeManchin
.csv opened
----------------------------------
Working on:   JoeManchinWV
.csv opened
----------------------------------
Working on:   SenMarkey
.csv opened
----------------------------------
Working on:   EdMarkey
.csv opened
----------------------------------
Working on:   SenatorMenendez
.csv opened
----------------------------------
Working on:   MenendezNJ
Not authorized.
Working on:   SenJeffMerkley
.csv opened
----------------------------------
Working on:   JeffMerkley
.csv opened
----------------------------------
Working on:   ChrisMurphyCT
.csv opened
----------------------------------
Working on:   senmurp

.csv opened
----------------------------------
Working on:   SenMikeLee
.csv opened
----------------------------------
Working on:   MikeLeeforUtah
.csv opened
----------------------------------
Working on:   SenatorLoeffler
.csv opened
----------------------------------
Working on:   KLoeffler
.csv opened
----------------------------------
Working on:   senatemajldr
.csv opened
----------------------------------
Working on:   Team_Mitch
.csv opened
----------------------------------
Working on:   SenMcSallyAZ
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Working on:   MarthaMcSally
.csv opened
----------------------------------
Working on:   JerryMoran
.csv opened
----------------------------------
Working on:   moranforkansas
.csv opened
----------------------------------
Working on:   LisaMurkowski
.csv opened
----------------------------------
Working on:   LisaForSenate
.csv opened
----------------------------------
Working on:   RandPaul
.csv opened
--------------